In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
# read csv
path = './data/'

def get_csv(path):
    # load CSV & drop zeros
    df = pd.read_csv(path+'driving_log.csv')
    df = df.drop(df[df['steering'] == 0.].index)
    df = df.drop(df[df['speed'] < 1.].index)
    return df

df = get_csv(path = path)

In [ ]:
# split
split_index = int((1-(1/11))*(len(df)))
df_train = df[:split_index]
df_val  = df[split_index:]

In [ ]:
# generate batch

#import sklearn
image_shape = cv2.imread( path+df['center'][df['center'].index[0]]).shape
batch_size = 5*7*7 # 3675 = 3*5*5*7*7

def get_batch(df,batch_size): 
    n_split = int(len(df)//batch_size)-1
    batch_idx = (df[:batch_size*n_split].index.values.reshape(n_split,batch_size,))
    while 1:
        for idx in batch_idx:
            batch_y      =      df['steering'][idx].values
            batch_x_path = path+df['center'  ][idx]
            batch_x      = np.empty(list([batch_size]) + list(image_shape))

            ii = 0
            for img_path in batch_x_path:
                image = cv2.imread(img_path)
                batch_x[ii] = image
                ii +=1

            assert(not(np.isnan((np.sum(batch_x)))))
            assert(not(np.isnan((np.sum(batch_y)))))
            yield (batch_x,batch_y) 

train_generator = get_batch(df_train, batch_size=batch_size)
validation_generator = get_batch(df_val, batch_size=batch_size)

In [ ]:
def visualise_dataset(image,label,steps):
    plt_num = 1
    print(image.shape)
    
    step_size = int(image.shape[0]/steps)
    
    #for image_idx in range(0,image.shape[0],step_size):
    for image_idx in range(0,image.shape[0],step_size):
        channels = image.shape[3]
        plt.figure(plt_num, figsize=(32,32))
        for channel in range(channels):
            plt.subplot(4,8, channel+1) # sets the number of feature maps to show on each row and column
            plt.title('channel ' + str(channel)) # displays the feature map number
            plt.imshow(image[image_idx,:,:, channel], interpolation="nearest", cmap="gray")
        plt.show()

        print('{1}, idx: {0}'.format(image_idx,label[image_idx],))
        
#%time visualise_dataset(batch_x,batch_y,steps=1)

In [ ]:
'''
def get_StratifiedShuffleSplit(batches_idx,batch_size):
    n_bins  = 5
    batch_size = 5*7*7
    y_classes = df['steering'].copy()*n_bins//1
    batches_idx = df.index
    data_size = len(batches_idx)
    n_split = int(data_size//batch_size)-1
    print(n_split)
    batch_range = df.index
    sss = StratifiedShuffleSplit(n_splits = n_split,test_size = 15)
    batch_idx, val_idx = next(sss.split((batch_range),(y_classes.values)))
    
    batch_idx = batch_idx[:batch_size*n_split].reshape((n_split,batch_size, ))
    batch_idx = batch_idx[:batch_size*n_split].reshape((n_split,batch_size, ))
    
    return batch_idx, val_idx

    batch_idx, val_idx = (get_batch_idx(batches_idx,batch_size = 256))
    batch_idx.shape
'''

In [ ]:
from keras.models import Sequential,load_model

from keras.layers.core import Dense, Activation, Dropout, Highway, Reshape, Flatten, Lambda
#from keras.layers.recurrent import LSTM, GRU, SimpleRNN
#from keras.layers.wrappers import TimeDistributed
#from keras.regularizers import l2, activity_l2

# Hyperparameter Compile
loss='mse' # 'hinge'
optimizer='Nadam' #'rmsprop'

# Hyperparameter Fit
nb_epoch= 22

#try:
#    model = load_model('model.h5')
#except:
def model_1():
        model = Sequential()
        model.add(Lambda(lambda x: x/127.5 - 1.,
            input_shape = image_shape,
            output_shape= image_shape))
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('linear'))
        return model

model = model_1()


%time model.compile(loss=loss, optimizer=optimizer)
model.fit_generator(generator         = train_generator, \
                    samples_per_epoch = batch_size, \
                    validation_data   = validation_generator, \
                    nb_val_samples    = 1, \
                    nb_epoch          = nb_epoch)

#train_generator, 
#samples_per_epoch= len(train_samples), 
#validation_data=validation_generator,
#nb_val_samples=len(validation_samples), nb_epoch=3)



try:
    model.save_weights("model.h5")
except:
    print("model.save_weights failed")
    
    

 